# KNN with scikit-learn - Lab

## Introduction

In this lab, you'll learn how to use scikit-learn's implementation of a KNN classifier on the classic Titanic dataset from Kaggle!
 

## Objectives

In this lab you will:

- Conduct a parameter search to find the optimal value for K 
- Use a KNN classifier to generate predictions on a real-world dataset 
- Evaluate the performance of a KNN model  


## Getting Started

Start by importing the dataset, stored in the `titanic.csv` file, and previewing it.

In [1]:
# Your code here
# Import pandas and set the standard alias 
import pandas as pd

# Import the data from 'titanic.csv' and store it in a pandas DataFrame 
raw_df = pd.read_csv("titanic.csv")

# Print the head of the DataFrame to ensure everything loaded correctly 
raw_df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


Great!  Next, you'll perform some preprocessing steps such as removing unnecessary columns and normalizing features.

## Preprocessing the data

Preprocessing is an essential component in any data science pipeline. It's not always the most glamorous task as might be an engaging data visual or impressive neural network, but cleaning and normalizing raw datasets is very essential to produce useful and insightful datasets that form the backbone of all data powered projects. This can include changing column types, as in: 


```python
df['col_name'] = df['col_name'].astype('int')
```
Or extracting subsets of information, such as: 

```python
import re
df['street'] = df['address'].map(lambda x: re.findall('(.*)?\n', x)[0])
```

> **Note:** While outside the scope of this particular lesson, **regular expressions** (mentioned above) are powerful tools for pattern matching! See the [regular expressions official documentation here](https://docs.python.org/3.6/library/re.html). 

Since you've done this before, you should be able to do this quite well yourself without much hand holding by now. In the cells below, complete the following steps:

1. Remove unnecessary columns (`'PassengerId'`, `'Name'`, `'Ticket'`, and `'Cabin'`) 
2. Convert `'Sex'` to a binary encoding, where female is `0` and male is `1` 
3. Detect and deal with any missing values in the dataset:  
    * For `'Age'`, replace missing values with the median age for the dataset  
    * For `'Embarked'`, drop the rows that contain missing values
4. One-hot encode categorical columns such as `'Embarked'` 
5. Store the target column, `'Survived'`, in a separate variable and remove it from the DataFrame  

While we always want to worry about data leakage, which is why we typically perform the split before the preprocessing, for this data set, we'll do some of the preprocessing first. The reason for this is that some of the values of the variables only have a handful of instances, and we want to make sure we don't lose any of them.

In [15]:
# Drop the unnecessary columns
df = raw_df.drop(["PassengerId","Name","Ticket","Cabin"], axis=1)
df.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,male,22.0,1,0,7.2500,S
1,1,1,female,38.0,1,0,71.2833,C
2,1,3,female,26.0,0,0,7.9250,S
3,1,1,female,35.0,1,0,53.1000,S
4,0,3,male,35.0,0,0,8.0500,S


In [16]:
# Convert Sex to binary encoding
df['Sex'] = df['Sex'].map({"female" : 0, "male" : 1 })
df.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,1,22.0,1,0,7.2500,S
1,1,1,0,38.0,1,0,71.2833,C
2,1,3,0,26.0,0,0,7.9250,S
3,1,1,0,35.0,1,0,53.1000,S
4,0,3,1,35.0,0,0,8.0500,S


In [17]:
# Find the number of missing values in each column
df.isna().sum()

Survived      0
Pclass        0
Sex           0
Age         177
SibSp         0
Parch         0
Fare          0
Embarked      2
dtype: int64

In [18]:
# Impute the missing values in 'Age'
df['Age'] = df['Age'].fillna(df.Age.median())
df.isna().sum()

Survived    0
Pclass      0
Sex         0
Age         0
SibSp       0
Parch       0
Fare        0
Embarked    2
dtype: int64

In [19]:
# Drop the rows missing values in the 'Embarked' column
df = df.dropna()
df.isna().sum()

Survived    0
Pclass      0
Sex         0
Age         0
SibSp       0
Parch       0
Fare        0
Embarked    0
dtype: int64

In [20]:
# One-hot encode the categorical columns
one_hot_df = pd.get_dummies(df)
one_hot_df.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked_C,Embarked_Q,Embarked_S
0,0,3,1,22.0,1,0,7.2500,0,0,1
1,1,1,0,38.0,1,0,71.2833,1,0,0
2,1,3,0,26.0,0,0,7.9250,0,0,1
3,1,1,0,35.0,1,0,53.1000,0,0,1
4,0,3,1,35.0,0,0,8.0500,0,0,1


In [21]:
# Assign the 'Survived' column to labels
labels = one_hot_df['Survived']

# Drop the 'Survived' column from one_hot_df
one_hot_df.drop('Survived',axis = 1, inplace =True)

## Create training and test sets

Now that you've preprocessed the data, it's time to split it into training and test sets. 

In the cell below:

* Import `train_test_split` from the `sklearn.model_selection` module 
* Use `train_test_split()` to split the data into training and test sets, with a `test_size` of `0.25`. Set the `random_state` to 42 

In [22]:
# Import train_test_split 
from sklearn.model_selection import train_test_split

# Split the data
X_train, X_test, y_train, y_test = train_test_split(one_hot_df,labels,test_size =0.25, random_state =42)

## Normalizing the data

The final step in your preprocessing efforts for this lab is to **_normalize_** the data. We normalize **after** splitting our data into training and test sets. This is to avoid information "leaking" from our test set into our training set (read more about data leakage [here](https://machinelearningmastery.com/data-leakage-machine-learning/) ). Remember that normalization (also sometimes called **_Standardization_** or **_Scaling_**) means making sure that all of your data is represented at the same scale. The most common way to do this is to convert all numerical values to z-scores. 

Since KNN is a distance-based classifier, if data is in different scales, then larger scaled features have a larger impact on the distance between points.

To scale your data, use `StandardScaler` found in the `sklearn.preprocessing` module. 

In the cell below:

* Import and instantiate `StandardScaler` 
* Use the scaler's `.fit_transform()` method to create a scaled version of the training dataset  
* Use the scaler's `.transform()` method to create a scaled version of the test dataset  
* The result returned by `.fit_transform()` and `.transform()` methods will be numpy arrays, not a pandas DataFrame. Create a new pandas DataFrame out of this object called `scaled_df`. To set the column names back to their original state, set the `columns` parameter to `one_hot_df.columns` 
* Print the head of `scaled_df` to ensure everything worked correctly 

In [23]:
# Import StandardScaler
from sklearn.preprocessing import StandardScaler

# Instantiate StandardScaler
scaler = StandardScaler()

# Transform the training and test sets
scaled_data_train = scaler.fit_transform(X_train)
scaled_data_test = scaler.transform(X_test)

# Convert into a DataFrame
scaled_df_train = pd.DataFrame(scaled_data_train,columns = one_hot_df.columns)
scaled_df_train.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked_C,Embarked_Q,Embarked_S
0,0.815528,-1.390655,-0.575676,-0.474917,-0.480663,-0.500108,-0.483046,-0.311768,0.620174
1,-0.386113,-1.390655,1.550175,-0.474917,-0.480663,-0.435393,-0.483046,-0.311768,0.620174
2,-0.386113,0.719086,-0.120137,-0.474917,-0.480663,-0.644473,-0.483046,-0.311768,0.620174
3,-1.587755,0.719086,-0.120137,-0.474917,-0.480663,-0.115799,-0.483046,-0.311768,0.620174
4,0.815528,-1.390655,-1.107139,0.413551,-0.480663,-0.356656,2.070197,-0.311768,-1.612452


You may have noticed that the scaler also scaled our binary/one-hot encoded columns, too! Although it doesn't look as pretty, this has no negative effect on the model. Each 1 and 0 have been replaced with corresponding decimal values, but each binary column still only contains 2 values, meaning the overall information content of each column has not changed.

## Fit a KNN model

Now that you've preprocessed the data it's time to train a KNN classifier and validate its accuracy. 

In the cells below:

* Import `KNeighborsClassifier` from the `sklearn.neighbors` module 
* Instantiate the classifier. For now, you can just use the default parameters  
* Fit the classifier to the training data/labels
* Use the classifier to generate predictions on the test data. Store these predictions inside the variable `test_preds` 

In [24]:
# Import KNeighborsClassifier
from sklearn.neighbors import KNeighborsClassifier

# Instantiate KNeighborsClassifier
clf = KNeighborsClassifier()

# Fit the classifier
clf.fit(scaled_data_train, y_train)

# Predict on the test set
test_preds = clf.predict(scaled_data_test)

## Evaluate the model

Now, in the cells below, import all the necessary evaluation metrics from `sklearn.metrics` and complete the `print_metrics()` function so that it prints out **_Precision, Recall, Accuracy, and F1-Score_** when given a set of `labels` (the true values) and `preds` (the models predictions). 

Finally, use `print_metrics()` to print the evaluation metrics for the test predictions stored in `test_preds`, and the corresponding labels in `y_test`. 

In [27]:
# Your code here 
# Import the necessary functions
from sklearn.metrics import precision_score, recall_score, accuracy_score, f1_score


In [28]:
# Complete the function
def print_metrics(labels, preds):
    print("Precision Score: {}".format(precision_score(labels,preds)))
    print("Recall Score: {}".format(recall_score(labels,preds)))
    print("Accuracy Score: {}".format(accuracy_score(labels,preds)))
    print("F1 Score: {}".format(f1_score(labels,preds)))
    
print_metrics(y_test, test_preds)

Precision Score: 0.7058823529411765
Recall Score: 0.7317073170731707
Accuracy Score: 0.7892376681614349
F1 Score: 0.718562874251497


> Interpret each of the metrics above, and explain what they tell you about your model's capabilities. If you had to pick one score to best describe the performance of the model, which would you choose? Explain your answer.

Write your answer below this line: 


________________________________________________________________________________

Precision Score: 0.706
Precision measures how many of the positive predictions made by the model are actually correct. A precision of 0.706 means ~70.6% of positive predictions are true positives.

Interpretation: The model has a decent precision, but it's not extremely high. This could indicate that some false positives are being predicted.
Recall Score: 0.732
Recall measures how many of the actual positive instances the model successfully identifies. A recall of 0.732 means ~73.2% of the actual positives are correctly predicted.

Interpretation: The model captures a reasonable proportion of the actual positives, though there is still room for improvement.
Accuracy Score: 0.789
Accuracy measures the overall proportion of correct predictions (true positives and true negatives) to the total number of predictions. An accuracy of 0.789 (~78.9%) is fairly good, but it may not reflect the model’s performance well if the data is imbalanced.

Interpretation: The model performs well overall, but accuracy might not tell the full story if there are more negatives than positives in the dataset.
F1 Score: 0.719
The F1 score is the harmonic mean of precision and recall, balancing the trade-off between the two. An F1 score of 0.719 (~71.9%) indicates a reasonable balance between precision and recall.

Interpretation: The model handles the precision-recall trade-off well, making it suitable for datasets where both metrics matter.
Overall Evaluation:
The model shows moderate performance across all metrics. While precision, recall, and F1 are in the 70% range, there is room for improvement, especially if the application requires higher reliability.
Strength: Balanced performance across precision, recall, and F1 score, with decent accuracy.
Weakness: Precision (0.706) slightly lags behind recall (0.732), indicating a few more false positives than desired.
When to Improve and How:
If false positives are critical (e.g., spam detection): Focus on improving precision by tuning the KNN parameters (e.g., increasing 
𝐾
K) or using feature selection.
If false negatives are critical (e.g., medical diagnosis): Focus on improving recall by experimenting with distance metrics or reducing dimensionality.
If you aim for a better balance, consider techniques like cross-validation, hyperparameter tuning, or testing a different algorithm.



If I had to pick one score to best describe the performance of the model, I would choose the F1 Score. Here's why:

Why F1 Score is the Best Choice:
Balances Precision and Recall:

The F1 score is the harmonic mean of precision and recall, providing a single metric that captures the trade-off between these two key measures.
This is especially important when the dataset might be imbalanced, or when both false positives and false negatives have significant consequences.
Avoids Misleading Interpretations of Accuracy:

Accuracy (0.789) might give the impression of strong performance, but it doesn’t account for class imbalances. For example, if one class dominates, a model can have high accuracy just by predicting the majority class.
Reflects the Model's Practical Usefulness:

In real-world applications, the ability to balance correctly identifying positives (recall) while minimizing false positives (precision) is often critical.
A good F1 score indicates the model is neither too lenient (low precision) nor too strict (low recall), making it a better summary metric than accuracy alone.
Example Contexts Where F1 Score is Vital:

Spam Detection: Avoiding too many false positives (precision) while catching actual spam (recall).
Medical Diagnosis: Balancing the need to correctly identify cases (recall) while minimizing unnecessary alarms (precision).
Why Not the Other Scores?
Precision (0.706): While it’s important, it only measures one aspect (how many positive predictions are correct), ignoring the ability to detect all positives.
Recall (0.732): Similarly, recall focuses solely on identifying positives, ignoring the trade-off with false positives.
Accuracy (0.789): High accuracy can be misleading, particularly if one class dominates the dataset (e.g., many more negatives than positives).
Conclusion:
The F1 score (0.719) is the most reliable single metric because it considers both precision and recall, making it a better indicator of the model's overall effectiveness in handling the classification task.

## Improve model performance

While your overall model results should be better than random chance, they're probably mediocre at best given that you haven't tuned the model yet. For the remainder of this notebook, you'll focus on improving your model's performance. Remember that modeling is an **_iterative process_**, and developing a baseline out of the box model such as the one above is always a good start. 

First, try to find the optimal number of neighbors to use for the classifier. To do this, complete the `find_best_k()` function below to iterate over multiple values of K and find the value of K that returns the best overall performance. 

The function takes in six arguments:
* `X_train`
* `y_train`
* `X_test`
* `y_test`
* `min_k` (default is 1)
* `max_k` (default is 25)
    
> **Pseudocode Hint**:
1. Create two variables, `best_k` and `best_score`
1. Iterate through every **_odd number_** between `min_k` and `max_k + 1`. 
    1. For each iteration:
        1. Create a new `KNN` classifier, and set the `n_neighbors` parameter to the current value for k, as determined by the loop 
        1. Fit this classifier to the training data 
        1. Generate predictions for `X_test` using the fitted classifier 
        1. Calculate the **_F1-score_** for these predictions 
        1. Compare this F1-score to `best_score`. If better, update `best_score` and `best_k` 
1. Once all iterations are complete, print the best value for k and the F1-score it achieved 

In [29]:
def find_best_k(X_train, y_train, X_test, y_test, min_k=1, max_k=25):
    # Your code here
    best_k = 0
    best_score = 0.0
    for k in range(min_k, max_k+1, 2):
        knn = KNeighborsClassifier(n_neighbors=k)
        knn.fit(X_train, y_train)
        preds = knn.predict(X_test)
        f1 = f1_score(y_test, preds)
        if f1 > best_score:
            best_k = k
            best_score = f1
    
    print("Best Value for k: {}".format(best_k))
    print("F1-Score: {}".format(best_score))


In [30]:
find_best_k(scaled_data_train, y_train, scaled_data_test, y_test)
# Expected Output:

# Best Value for k: 17
# F1-Score: 0.7468354430379746

Best Value for k: 17
F1-Score: 0.7468354430379746


If all went well, you'll notice that model performance has improved by 3 percent by finding an optimal value for k. For further tuning, you can use scikit-learn's built-in `GridSearch()` to perform a similar exhaustive check of hyperparameter combinations and fine tune model performance. For a full list of model parameters, see the [sklearn documentation !](https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html)



## (Optional) Level Up: Iterating on the data

As an optional (but recommended!) exercise, think about the decisions you made during the preprocessing steps that could have affected the overall model performance. For instance, you were asked to replace the missing age values with the column median. Could this have affected the overall performance? How might the model have fared if you had just dropped those rows, instead of using the column median? What if you reduced the data's dimensionality by ignoring some less important columns altogether?

In the cells below, revisit your preprocessing stage and see if you can improve the overall results of the classifier by doing things differently. Consider dropping certain columns, dealing with missing values differently, or using an alternative scaling function. Then see how these different preprocessing techniques affect the performance of the model. Remember that the `find_best_k()` function handles all of the fitting; use this to iterate quickly as you try different strategies for dealing with data preprocessing! 

In [31]:
# Drop the unnecessary columns
df_2 = raw_df.drop(["PassengerId","Name","Ticket","Cabin"], axis=1)
df_2.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,male,22.0,1,0,7.2500,S
1,1,1,female,38.0,1,0,71.2833,C
2,1,3,female,26.0,0,0,7.9250,S
3,1,1,female,35.0,1,0,53.1000,S
4,0,3,male,35.0,0,0,8.0500,S


In [32]:
# Convert Sex to binary encoding
df_2['Sex'] = df_2['Sex'].map({"female" : 0, "male" : 1 })
df_2.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,1,22.0,1,0,7.2500,S
1,1,1,0,38.0,1,0,71.2833,C
2,1,3,0,26.0,0,0,7.9250,S
3,1,1,0,35.0,1,0,53.1000,S
4,0,3,1,35.0,0,0,8.0500,S


In [33]:
df_2.isna().sum()

Survived      0
Pclass        0
Sex           0
Age         177
SibSp         0
Parch         0
Fare          0
Embarked      2
dtype: int64

In [34]:
# Drop the rows missing values in the 'Embarked' column
df_2 = df_2.dropna()
df_2.isna().sum()

Survived    0
Pclass      0
Sex         0
Age         0
SibSp       0
Parch       0
Fare        0
Embarked    0
dtype: int64

In [35]:
# One-hot encode the categorical columns
one_hot_df_2 = pd.get_dummies(df_2)
one_hot_df_2.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked_C,Embarked_Q,Embarked_S
0,0,3,1,22.0,1,0,7.2500,0,0,1
1,1,1,0,38.0,1,0,71.2833,1,0,0
2,1,3,0,26.0,0,0,7.9250,0,0,1
3,1,1,0,35.0,1,0,53.1000,0,0,1
4,0,3,1,35.0,0,0,8.0500,0,0,1


In [36]:
# Assign the 'Survived' column to labels
labels_2 = one_hot_df_2['Survived']

# Drop the 'Survived' column from one_hot_df
one_hot_df_2.drop('Survived',axis = 1, inplace =True)

In [42]:
# Import train_test_split 
from sklearn.model_selection import train_test_split

# Split the data
X_train, X_test, y_train, y_test = train_test_split(one_hot_df_2,labels_2,test_size =0.25, random_state =42)

In [38]:
find_best_k(X_train, y_train, X_test, y_test)

Best Value for k: 3
F1-Score: 0.6341463414634146


In [47]:
# Import StandardScaler
from sklearn.preprocessing import StandardScaler

# Instantiate StandardScaler
scaler = StandardScaler()

# Transform the training and test sets
scaled_data_train_2 = scaler.fit_transform(X_train)
scaled_data_test_2 = scaler.transform(X_test)

# Convert into a DataFrame
scaled_df_train_2 = pd.DataFrame(scaled_data_train_2,columns = one_hot_df_2.columns)
scaled_df_train_2.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked_C,Embarked_Q,Embarked_S
0,0.887658,0.746225,-0.320248,-0.540675,-0.475762,-0.510641,-0.453230,5.354126,-1.974609
1,-1.512390,0.746225,1.211827,-0.540675,-0.475762,0.144234,-0.453230,-0.186772,0.506429
2,0.887658,-1.340079,-0.389887,-0.540675,3.119385,-0.265441,2.206384,-0.186772,-1.974609
3,0.887658,0.746225,0.167231,-0.540675,-0.475762,-0.504077,-0.453230,-0.186772,0.506429
4,0.887658,0.746225,0.654709,-0.540675,-0.475762,-0.161292,-0.453230,-0.186772,0.506429


In [48]:
find_best_k(scaled_data_train_2 , y_train,scaled_data_test_2, y_test)

Best Value for k: 7
F1-Score: 0.7435897435897437


In [49]:
# Import MinMaxScaler
from sklearn.preprocessing import MinMaxScaler

# Instantiate StandardScaler
scaler_2 = MinMaxScaler()

# Transform the training and test sets
scaled_data_train_3 = scaler_2.fit_transform(X_train)
scaled_data_test_3 = scaler_2.transform(X_test)

# Convert into a DataFrame
scaled_df_train_3 = pd.DataFrame(scaled_data_train_3,columns = one_hot_df_2.columns)
scaled_df_train_3.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked_C,Embarked_Q,Embarked_S
0,1.0,1.0,0.308872,0.0,0.0,0.015111,0.0,1.0,0.0
1,0.0,1.0,0.585323,0.0,0.0,0.075147,0.0,0.0,1.0
2,1.0,0.0,0.296306,0.0,0.5,0.037590,1.0,0.0,0.0
3,1.0,1.0,0.396833,0.0,0.0,0.015713,0.0,0.0,1.0
4,1.0,1.0,0.484795,0.0,0.0,0.047138,0.0,0.0,1.0


In [50]:
find_best_k(scaled_data_train_3 , y_train,scaled_data_test_3, y_test)

Best Value for k: 19
F1-Score: 0.7412587412587414


In [52]:
# Drop the unnecessary columns
df_3 = raw_df.drop(["PassengerId","Name","Ticket","Cabin","SibSp","Parch"], axis=1)
df_3.head()

,Survived,Pclass,Sex,Age,Fare,Embarked
0,0,3,male,22.0,7.2500,S
1,1,1,female,38.0,71.2833,C
2,1,3,female,26.0,7.9250,S
3,1,1,female,35.0,53.1000,S
4,0,3,male,35.0,8.0500,S


In [53]:
# Convert Sex to binary encoding
df_3['Sex'] = df_3['Sex'].map({"female" : 0, "male" : 1 })
df_3.head()

,Survived,Pclass,Sex,Age,Fare,Embarked
0,0,3,1,22.0,7.2500,S
1,1,1,0,38.0,71.2833,C
2,1,3,0,26.0,7.9250,S
3,1,1,0,35.0,53.1000,S
4,0,3,1,35.0,8.0500,S


In [54]:
# Impute the missing values in 'Age'
df_3['Age'] = df_3['Age'].fillna(df_3.Age.median())
df_3.isna().sum()

Survived    0
Pclass      0
Sex         0
Age         0
Fare        0
Embarked    2
dtype: int64

In [55]:
df_3 = df_3.dropna()
df_3.isna().sum()

Survived    0
Pclass      0
Sex         0
Age         0
Fare        0
Embarked    0
dtype: int64

In [56]:
# One-hot encode the categorical columns
one_hot_df_3 = pd.get_dummies(df_3)
one_hot_df_3.head()

,Survived,Pclass,Sex,Age,Fare,Embarked_C,Embarked_Q,Embarked_S
0,0,3,1,22.0,7.2500,0,0,1
1,1,1,0,38.0,71.2833,1,0,0
2,1,3,0,26.0,7.9250,0,0,1
3,1,1,0,35.0,53.1000,0,0,1
4,0,3,1,35.0,8.0500,0,0,1


In [57]:
# Assign the 'Survived' column to labels
labels_3 = one_hot_df_3['Survived']

# Drop the 'Survived' column from one_hot_df
one_hot_df_3.drop('Survived',axis = 1, inplace =True)

In [58]:
# Import train_test_split 
from sklearn.model_selection import train_test_split

# Split the data
X_train, X_test, y_train, y_test = train_test_split(one_hot_df_3,labels_3,test_size =0.25, random_state =42)

In [59]:
# Import StandardScaler
from sklearn.preprocessing import StandardScaler

# Instantiate StandardScaler
scaler = StandardScaler()

# Transform the training and test sets
scaled_data_train_3 = scaler.fit_transform(X_train)
scaled_data_test_3 = scaler.transform(X_test)

# Convert into a DataFrame
scaled_df_train_3= pd.DataFrame(scaled_data_train_3,columns = one_hot_df_3.columns)
scaled_df_train_3.head()

,Pclass,Sex,Age,Fare,Embarked_C,Embarked_Q,Embarked_S
0,0.815528,-1.390655,-0.575676,-0.500108,-0.483046,-0.311768,0.620174
1,-0.386113,-1.390655,1.550175,-0.435393,-0.483046,-0.311768,0.620174
2,-0.386113,0.719086,-0.120137,-0.644473,-0.483046,-0.311768,0.620174
3,-1.587755,0.719086,-0.120137,-0.115799,-0.483046,-0.311768,0.620174
4,0.815528,-1.390655,-1.107139,-0.356656,2.070197,-0.311768,-1.612452


In [60]:
find_best_k(scaled_data_train_3, y_train, scaled_data_test_3, y_test)

Best Value for k: 11
F1-Score: 0.7643312101910827


## Summary

Well done! In this lab, you worked with the classic Titanic dataset and practiced fitting and tuning KNN classification models using scikit-learn! As always, this gave you another opportunity to continue practicing your data wrangling skills and model tuning skills using Pandas and scikit-learn!